In [1]:
import numpy as np
from scipy.optimize import root_scalar
from scipy.optimize import brentq
import matplotlib.pyplot as plt

In [24]:
#parameters
t = 0.425
e=1.602*10**(-19)
kB = 8.617e-5
T=0.1
beta=1/(kB*T)

A0=0.1
Theta=0
V=1


# k-grid
n_k = 150
kx = np.linspace(-np.pi, np.pi, n_k)
ky = np.linspace(-np.pi, np.pi, n_k)
KX, KY = np.meshgrid(kx, ky)

In [2]:
def formfactor(kx,ky):
    return np.cos(kx) - np.cos(ky)


In [21]:
def safe_divide(a,b):
    out = np.zeros_like(a)
    mask = (b > 1e-12)
    out[mask] = a[mask]/b[mask]
    return out


In [3]:
def epsilon(t,kx,ky,mu):
    return -2*t*(np.cos(kx)+np.cos(ky)) - mu

def Ek(t,kx,ky,mu,delta):
    return np.sqrt(epsilon(t,kx,ky,mu)**2 + delta**2 * formfactor(kx,ky)**2)


In [15]:
def filling(mu,delta):
    xi = epsilon(t,KX,KY,mu)
    E  = Ek(t,KX,KY,mu,delta)

    ratio = safe_divide(xi,E)
    return np.mean(1 - ratio*np.tanh(beta*E/2))


In [16]:
def filling_normal(mu):
    xi = epsilon(t,KX,KY,mu)
    return np.mean(2*(xi < 0))


In [17]:
def gap_rhs(mu,delta):
    fk = formfactor(KX,KY)
    E  = Ek(t,KX,KY,mu,delta)

    return np.mean(fk**2 * safe_divide(1.0,2*E) * np.tanh(beta*E/2))



In [18]:
from scipy.optimize import brentq

def mu_at_delta0():
    f = lambda mu: filling_normal(mu) - 1.0
    return brentq(f,-4*t,4*t)



In [25]:
def solve_gap():

    mu = mu_at_delta0()   # safe normal-state μ

    delta = 0.1*t
    mix = 0.5

    for it in range(200):

        # Step 2: enforce filling for Δ>0
        f = lambda mu: filling(mu,delta) - 1.0
        mu = brentq(f,-4*t,4*t)

        # Step 3: gap update
        new_delta = abs(V) * gap_rhs(mu,delta)

        delta = mix*new_delta + (1-mix)*delta

        if abs(new_delta-delta)/delta < 1e-6:
            break

    return delta,mu


In [26]:
solve_gap()

TypeError: 'float' object is not subscriptable